In [2]:
import pandas as pd
import numpy as np

# read dog dataset, No is encoded as 0, Yes as 1, 
# bite refers to class 1, bark to class 0

data = pd.read_csv("dogs.csv") 
print(data)

   Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
2      1       1    0         1       0
3      1       0    0         1       1
4      0       1    1         0       1
5      0       0    1         1       1
6      0       0    0         1       1
7      1       1    0         0       1


You can access individual columns by name to get a pandas series or convert it to a NumPy array:

In [ ]:
print(data["Heavy"])
print(data["Heavy"].to_numpy())

### What is the entropy of the target value 'Action' in the whole dataset? 
Define a function that calculates the entropy (for a list of 0/1 values):

In [ ]:
def entropy(values):
    return None

Apply it to the "Action" column

In [ ]:
print(entropy(data["Action"].to_numpy()))

### Which attribute would the ID3 algorithm choose to use for the root of the tree? What is its information gain? 

Iterate through all categories, make the potential split, compare the resulting entropies and take the greedy choice. You might need:

```
# iterating the columns 
for col in data.columns: 
    print(col)
```

and to filter entries of a dataframe that satisfy a certain condition, you might want to use:
```
data[data["Heavy"] == 0] # only those entries that are not heavy
```

###  Draw the full decision tree that would be learned for this data using ID3 without pruning.

Recursively apply the splitting procedure until all nodes are leaf nodes (= pure in the target). You may use the class `Node` as a start for your implementation and attributes as needed.

In [ ]:
class Node:
    
    def entropy(self, values):
        # could be empty  while trying for a split
        if values.size == 0:
            return 0
        
        ratio_ones = sum(values) / len(values)
        ratio_zeros = 1 - ratio_ones

        pd = [ratio_ones, ratio_zeros]
        if ratio_ones == 0 or ratio_zeros == 0:
            return 0
        return - sum( [ p_i * np.log2(p_i) for p_i in pd] )
    
    def __init__(self, data, ancestor_features):
        # pure if there are all ones or all zeros 
        self.pure = sum(data["Action"]) == len(data) or  sum(data["Action"]) == 0
        self.split_feature = None
        self.data = data
        self.base_entropy = self.entropy(data["Action"].to_numpy())
        self.children = None
        # since we're using categorical (binary), each features should at most be used once in any branch
        self.ancestor_features = ancestor_features 
        # You can use these ancestor_features to get a nice identation:
        # print(" "*len(self.ancestor_features) + f"Initializing with data {data}")
        
    def split(self):
        # implement the split here
        
    def train(self):        
        if not self.pure:
            self.split()
            for child in self.children:
                # assume that we can get nodes pure by looking at our features
                # having exactly the same features but two different classes would not work here
                child.train()
                # now child is either pure or split into nodes that are eventually pure

# start training with a root node consisting of all data
root = Node(data, [])
root.train()

### Test on three new dogs
Here's the test set, evaluate the predicted "Action" classes using your implementation and compare them to the true "Action" classes.

In [3]:
data = pd.read_csv("dogs_test.csv") 
print(data)

   Heavy  Smelly  Big  Growling  Action
0      1       0    1         1       0
1      0       0    0         1       0
2      1       1    0         0       1


### Repeat with the Gini coefficient
Just alter your previous implementations 